<a href="https://colab.research.google.com/github/spencer18001/ml_zoomcamp_project_2024_midterm/blob/main/Notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%%capture
!pip install tqdm
!wget -O player_statistics_cleaned_final.csv https://raw.githubusercontent.com/spencer18001/ml_zoomcamp_project_2024_midterm/refs/heads/main/Data/player_statistics_cleaned_final.csv

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mutual_info_score, root_mean_squared_error
from tqdm.auto import tqdm

## EDA

In [3]:
df_path = "player_statistics_cleaned_final.csv"
df = pd.read_csv(df_path)
df

,TeamName,PlayerName,Position,Games,Win rate,KDA,Avg kills,Avg deaths,Avg assists,CSPerMin,...,Avg VWPM,GD@15,CSD@15,XPD@15,FB %,FB Victim,Penta Kills,Solo Kills,Country,FlashKeybind
0,Top Esports,369,Top,8,0.500,3.1,2.5,2.4,4.9,7.6,...,0.13,322,5,424,0.000,0.250,0,2,China,D
1,Dplus KIA,aiming,Adc,9,0.333,4.8,5.0,2.0,4.7,10.1,...,0.10,-112,1,97,0.333,0.000,0,2,South Korea,F
2,MAD Lions KOI,alvaro,Support,5,0.200,1.5,0.2,4.4,6.2,1.2,...,0.49,-3,2,-256,0.000,0.600,0,-,Spain,D
3,Team Liquid,apa,Mid,10,0.500,2.4,3.5,3.2,4.2,8.7,...,0.18,278,2,216,0.400,0.000,0,3,United States,D
4,PSG Talon,azhi,Top,5,0.200,2.3,2.2,3.0,4.8,7.5,...,0.18,-311,-10,-168,0.200,0.000,0,-,Taiwan,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Team Liquid,yeon,Adc,10,0.500,4.9,3.8,1.8,5.0,10.0,...,0.16,224,6,571,0.500,0.000,0,-,United States,D
77,G2 Esports,yike,Jungle,8,0.375,4.5,3.1,2.4,7.6,6.4,...,0.22,102,4,-80,0.250,0.000,0,2,Sweden,D
78,Hanwha Life Esports,zeka,Mid,12,0.500,3.6,3.8,2.8,6.2,9.1,...,0.24,288,2,66,0.250,0.167,0,1,South Korea,F
79,T1,zeus,Top,8,0.875,7.1,3.9,1.6,7.6,7.9,...,0.19,633,8,370,0.125,0.000,0,2,South Korea,F


In [4]:
df.isnull().sum()

,0
TeamName,0
PlayerName,0
Position,0
Games,0
Win rate,0
KDA,0
Avg kills,0
Avg deaths,0
Avg assists,0
CSPerMin,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TeamName       81 non-null     object 
 1   PlayerName     81 non-null     object 
 2   Position       81 non-null     object 
 3   Games          81 non-null     int64  
 4   Win rate       81 non-null     float64
 5   KDA            81 non-null     float64
 6   Avg kills      81 non-null     float64
 7   Avg deaths     81 non-null     float64
 8   Avg assists    81 non-null     float64
 9   CSPerMin       81 non-null     float64
 10  GoldPerMin     81 non-null     int64  
 11  KP%            81 non-null     float64
 12  DamagePercent  81 non-null     float64
 13  DPM            81 non-null     int64  
 14  VSPM           81 non-null     float64
 15  Avg WPM        81 non-null     float64
 16  Avg WCPM       81 non-null     float64
 17  Avg VWPM       81 non-null     float64
 18  GD@15       

In [6]:
for c in df.dtypes[df.dtypes == 'object'].index:
    print(f"{df[c].value_counts().head(8)}\n")

TeamName
Bilibili Gaming    6
Top Esports        5
Dplus KIA          5
MAD Lions KOI      5
Team Liquid        5
PSG Talon          5
Weibo Gaming       5
G2 Esports         5
Name: count, dtype: int64

PlayerName
369          1
kingen       1
peyz         1
peanut       1
oscarinin    1
oner         1
on           1
noah         1
Name: count, dtype: int64

Position
Jungle     17
Top        16
Adc        16
Support    16
Mid        16
Name: count, dtype: int64

Solo Kills
-    38
1    18
2    16
3     6
4     1
6     1
5     1
Name: count, dtype: int64

Country
South Korea      30
China            19
Spain             6
Taiwan            5
Vietnam           5
United States     3
Brazil            3
Poland            2
Name: count, dtype: int64

FlashKeybind
D    44
F    37
Name: count, dtype: int64



In [7]:
df["Solo Kills"] = df["Solo Kills"].replace(to_replace="-", value=0)
df["Solo Kills"] = df["Solo Kills"].astype(int)
df["Solo Kills"].dtype

dtype('int64')

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Games,81.0,8.641975,2.908215,3.000,6.000,8.000,10.000,14.000
Win rate,81.0,0.455975,0.214833,0.000,0.333,0.500,0.615,0.875
KDA,81.0,3.765432,1.834677,1.200,2.400,3.400,4.400,9.300
Avg kills,81.0,2.686420,1.519355,0.200,1.200,2.500,3.900,5.600
Avg deaths,81.0,2.875309,0.896734,1.300,2.400,2.900,3.300,5.600
Avg assists,81.0,6.755556,2.390293,1.800,4.900,6.200,8.300,12.400
CSPerMin,81.0,6.656790,3.048850,1.000,5.700,7.900,8.900,10.500
GoldPerMin,81.0,362.654321,72.095797,225.000,320.000,385.000,416.000,486.000
KP%,81.0,0.680790,0.088962,0.410,0.623,0.691,0.751,0.849
DamagePercent,81.0,0.199605,0.084498,0.052,0.135,0.215,0.275,0.337


In [9]:
df.drop(columns=["Penta Kills"], inplace=True)

In [10]:
X = df.drop(columns=["Win rate"])
y = df["Win rate"]

encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X)

mi = mutual_info_regression(X_encoded, y)

mi_lst = list(zip(X.columns, mi))
sorted_mi_lst = sorted(mi_lst, key=lambda x: x[1], reverse=True)
sorted_mi_lst

[('TeamName', 2.1456974159920006),
 ('Games', 1.4816193074760697),
 ('Country', 0.6853233597071364),
 ('KDA', 0.44318055338363305),
 ('FB %', 0.2506083147075029),
 ('GD@15', 0.21641249214643565),
 ('Avg assists', 0.17453110029625485),
 ('Avg deaths', 0.16685722588575036),
 ('XPD@15', 0.14482679248110042),
 ('FB Victim', 0.11944757665421113),
 ('DPM', 0.05755853247479337),
 ('Avg WPM', 0.056423519970109304),
 ('CSD@15', 0.048913483794382984),
 ('FlashKeybind', 0.03773882805240536),
 ('KP%', 0.03327653867786795),
 ('Avg VWPM', 0.016700753316256645),
 ('VSPM', 0.008262033570668681),
 ('PlayerName', 0.0),
 ('Position', 0.0),
 ('Avg kills', 0.0),
 ('CSPerMin', 0.0),
 ('GoldPerMin', 0.0),
 ('DamagePercent', 0.0),
 ('Avg WCPM', 0.0),
 ('Solo Kills', 0.0)]

In [11]:
df.drop(columns=["TeamName", "PlayerName", "Country"], inplace=True)

In [12]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

df_full_train = df_full_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_full_train = df_full_train["Win rate"].values
y_test = df_test["Win rate"].values

del df_full_train["Win rate"]
del df_test["Win rate"]

In [13]:
dv = DictVectorizer(sparse=False)

full_train_dict = df_full_train.to_dict(orient='records')
X_full_train = dv.fit_transform(full_train_dict)

test_dict = df_test.to_dict(orient='records')
X_test = dv.fit_transform(test_dict)

## Linear Regression

In [28]:
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

def train_kfold(train_func):
    rmse_list = []
    for train_idx, val_idx in kfold.split(df_full_train):
        X_train = X_full_train[train_idx]
        X_val = X_full_train[val_idx]

        y_train = y_full_train[train_idx]
        y_val = y_full_train[val_idx]

        y_pred = train_func(X_train, y_train, X_val)

        rmse = root_mean_squared_error(y_val, y_pred)
        rmse_list.append(rmse)
    return rmse_list

In [29]:
def train_ridge(X_train, y_train, X_val):
    model = Ridge(alpha=alpha, random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return y_pred

In [30]:
for alpha in tqdm([0, 0.01, 0.1, 0.5, 1, 10, 100]):
    rmse_list = train_kfold(train_func=train_ridge)

    print('alpha=%-5s %.3f +- %.3f' % (alpha, np.mean(rmse_list), np.std(rmse_list)))

  0%|          | 0/7 [00:00<?, ?it/s]

alpha=0     0.100 +- 0.009
alpha=0.01  0.095 +- 0.011
alpha=0.1   0.095 +- 0.008
alpha=0.5   0.098 +- 0.009
alpha=1     0.099 +- 0.011
alpha=10    0.101 +- 0.021
alpha=100   0.113 +- 0.027
